# Multi-Head Attention\n\n## Introduction\n\nBienvenue dans ce notebook sur le **Multi-Head Attention**, le mécanisme qui permet aux transformers de capturer différents types de relations en parallèle!\n\n### Objectifs pédagogiques\n\nDans ce notebook, vous allez:\n1. Comprendre le concept de multi-head attention\n2. Implémenter multi-head attention from scratch avec NumPy\n3. Implémenter multi-head attention avec PyTorch\n4. Visualiser les différentes têtes d'attention\n5. Comprendre comment les têtes se spécialisent\n6. Expérimenter avec différents nombres de têtes\n\n### Qu'est-ce que le Multi-Head Attention?\n\nAu lieu d'une seule attention, on utilise plusieurs **têtes** (heads) qui regardent différents aspects des relations entre tokens.\n\n**Analogie:** Imaginez que vous lisez un texte avec plusieurs perspectives:\n- **Tête 1:** Relations grammaticales (sujet-verbe, déterminant-nom)\n- **Tête 2:** Relations sémantiques (coréférences, synonymes)\n- **Tête 3:** Relations positionnelles (proximité, distance)\n\nChaque tête peut se spécialiser dans un type de relation différent!

## 1. Formules Mathématiques du Multi-Head Attention\n\n### Formule Complète\n\n$\\text{MultiHead}(Q, K, V) = \\text{Concat}(\\text{head}_1, ..., \\text{head}_h)W^O$\n\nOù chaque tête est définie par:\n\n$\\text{head}_i = \\text{Attention}(QW^Q_i, KW^K_i, VW^V_i)$\n\n### Décomposition Étape par Étape\n\n#### Étape 1: Projections Linéaires\n\n$Q = XW^Q, \\quad K = XW^K, \\quad V = XW^V$\n\nOù:\n- $X \\in \\mathbb{R}^{n \\times d_{\\text{model}}}$ : Input embeddings\n- $W^Q, W^K, W^V \\in \\mathbb{R}^{d_{\\text{model}} \\times d_{\\text{model}}}$ : Matrices de projection\n- $Q, K, V \\in \\mathbb{R}^{n \\times d_{\\text{model}}}$ : Queries, Keys, Values projetées\n\n#### Étape 2: Division en Têtes\n\n$d_k = \\frac{d_{\\text{model}}}{h}$\n\nOù $h$ est le nombre de têtes.\n\n**Reshape:**\n$(\\text{batch}, n, d_{\\text{model}}) \\rightarrow (\\text{batch}, n, h, d_k) \\rightarrow (\\text{batch}, h, n, d_k)$\n\n#### Étape 3: Attention par Tête\n\nPour chaque tête $i$:\n\n$\\text{head}_i = \\text{softmax}\\left(\\frac{Q_i K_i^T}{\\sqrt{d_k}}\\right)V_i$\n\nOù $Q_i, K_i, V_i \\in \\mathbb{R}^{n \\times d_k}$\n\n#### Étape 4: Concaténation\n\n$\\text{Concat}(\\text{head}_1, ..., \\text{head}_h) = [\\text{head}_1; \\text{head}_2; ...; \\text{head}_h]$\n\n**Dimensions:**\n- Avant concat: $h$ têtes de dimension $d_k$\n- Après concat: $h \\times d_k = d_{\\text{model}}$\n\n#### Étape 5: Projection Finale\n\n$\\text{Output} = \\text{Concat}(\\text{heads})W^O$\n\nOù $W^O \\in \\mathbb{R}^{d_{\\text{model}} \\times d_{\\text{model}}}$\n\n### Pourquoi Multi-Head?\n\n**Avantages:**\n1. **Diversité:** Chaque tête peut se spécialiser dans différents patterns\n2. **Parallélisme:** Toutes les têtes calculent en parallèle\n3. **Capacité:** Plus de paramètres = plus de capacité d'apprentissage\n4. **Robustesse:** Si une tête échoue, les autres compensent

In [ ]:
# Imports\nimport numpy as np\nimport math\nimport torch\nimport torch.nn as nn\nimport torch.nn.functional as F\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nimport sys\nfrom typing import Tuple, Optional\n\n# Ajouter le chemin vers src/\nsys.path.append('../..')\n\n# Importer nos modules\nfrom src.attention.multi_head import (\n    multi_head_attention_from_scratch,\n    MultiHeadAttention\n)\nfrom src.attention.scaled_dot_product import (\n    scaled_dot_product_attention_from_scratch,\n    ScaledDotProductAttention\n)\n\n# Configuration pour les visualisations\nplt.style.use('default')\nsns.set_palette(\"husl\")\n\n# Seed pour la reproductibilité\nnp.random.seed(42)\ntorch.manual_seed(42)\n\nprint(\"✓ Imports réussis!\")\nprint(f\"✓ PyTorch version: {torch.__version__}\")\nprint(f\"✓ Device disponible: {'GPU' if torch.cuda.is_available() else 'CPU'}\")

## 2. Implémentation From Scratch (NumPy)\n\nCommençons par implémenter multi-head attention avec NumPy pour comprendre chaque opération.\n\n### 2.1 Exemple avec 3 Tokens et 2 Têtes

In [ ]:
# Paramètres pour l'exemple\nbatch_size = 1\nseq_len = 3      # 3 tokens\nd_model = 8      # dimension du modèle\nnum_heads = 2    # 2 têtes d'attention\n\n# Créer l'input\nnp.random.seed(42)\nx = np.random.randn(batch_size, seq_len, d_model)\n\n# Créer les matrices de projection\nW_q = np.random.randn(d_model, d_model) * 0.1\nW_k = np.random.randn(d_model, d_model) * 0.1\nW_v = np.random.randn(d_model, d_model) * 0.1\nW_o = np.random.randn(d_model, d_model) * 0.1\n\nprint(\"=\" * 70)\nprint(\"EXEMPLE: Multi-Head Attention from Scratch\")\nprint(\"=\" * 70)\nprint(f\"\\nConfiguration:\")\nprint(f\"  - batch_size: {batch_size}\")\nprint(f\"  - seq_len: {seq_len}\")\nprint(f\"  - d_model: {d_model}\")\nprint(f\"  - num_heads: {num_heads}\")\nprint(f\"  - d_k (par tête): {d_model // num_heads}\")

In [ ]:
# Calculer multi-head attention from scratch\noutput, attention_weights = multi_head_attention_from_scratch(\n    x, W_q, W_k, W_v, W_o, num_heads\n)\n\nprint(\"\\n\" + \"=\" * 70)\nprint(\"RÉSULTATS\")\nprint(\"=\" * 70)\nprint(f\"\\nOutput shape: {output.shape}\")\nprint(f\"Nombre de têtes: {len(attention_weights)}\")\n\nprint(f\"\\nPoids d'attention par tête:\")\nfor i, weights in enumerate(attention_weights):\n    print(f\"\\n  Tête {i+1}:\")\n    print(f\"    Shape: {weights.shape}\")\n    print(f\"    Poids (batch 0):\")\n    print(f\"{weights[0]}\")\n    \n    # Vérifier la normalisation\n    row_sums = weights[0].sum(axis=-1)\n    print(f\"    Somme par ligne: {row_sums}\")\n    print(f\"    Normalisé? {np.allclose(row_sums, 1.0)}\")

## 3. Implémentation PyTorch (Professionnelle)\n\nMaintenant, utilisons PyTorch pour une implémentation optimisée et parallèle.\n\n### 3.1 Classe MultiHeadAttention

In [ ]:
# Créer des tensors PyTorch\ntorch.manual_seed(42)\nx_torch = torch.randn(batch_size, seq_len, d_model)\n\nprint(\"=\" * 70)\nprint(\"EXEMPLE: Multi-Head Attention avec PyTorch\")\nprint(\"=\" * 70)\nprint(f\"\\nInput shape: {x_torch.shape}\")\nprint(f\"Device: {x_torch.device}\")\n\n# Créer le module Multi-Head Attention\nmha = MultiHeadAttention(d_model=d_model, num_heads=num_heads)\n\nprint(f\"\\nModèle créé:\")\nprint(f\"  - Paramètres: {sum(p.numel() for p in mha.parameters())}\")\n\n# Calculer multi-head attention\noutput_torch = mha(x_torch)\n\nprint(f\"\\nOutput shape: {output_torch.shape}\")\nprint(f\"Dimension préservée? {x_torch.shape == output_torch.shape}\")

## 4. Visualisation des Têtes d'Attention\n\nVisualisons les poids d'attention de chaque tête pour comprendre comment elles se spécialisent.

In [ ]:
# Extraire les poids d'attention\nattention_weights_torch = mha.get_attention_weights(x_torch)\n\n# Visualiser chaque tête\ntokens = [\"Token 0\", \"Token 1\", \"Token 2\"]\n\nfig, axes = plt.subplots(1, num_heads, figsize=(12, 4))\n\nfor i in range(num_heads):\n    ax = axes[i] if num_heads > 1 else axes\n    \n    sns.heatmap(\n        attention_weights_torch[0, i].detach().numpy(),\n        ax=ax,\n        xticklabels=tokens,\n        yticklabels=tokens,\n        cmap='YlOrRd',\n        annot=True,\n        fmt='.3f',\n        cbar_kws={'label': 'Poids'},\n        vmin=0,\n        vmax=1\n    )\n    ax.set_title(f'Tête {i+1}', fontweight='bold')\n    ax.set_xlabel('Keys')\n    ax.set_ylabel('Queries')\n\nplt.tight_layout()\nplt.show()\n\nprint(\"\\nObservation:\")\nprint(\"  - Chaque tête a des patterns d'attention différents\")\nprint(\"  - Les têtes peuvent se spécialiser dans différents types de relations\")

## 5. Vérification des Dimensions (Shape Checks)\n\nVérifions que les dimensions sont correctes à chaque étape.

In [ ]:
print(\"=\" * 70)\nprint(\"VÉRIFICATION DES DIMENSIONS\")\nprint(\"=\" * 70)\n\n# Test avec différentes configurations\ntest_configs = [\n    (2, 4, 16, 2),   # batch=2, seq=4, d_model=16, heads=2\n    (1, 8, 32, 4),   # batch=1, seq=8, d_model=32, heads=4\n    (4, 10, 64, 8),  # batch=4, seq=10, d_model=64, heads=8\n]\n\nfor batch, seq, d_model, heads in test_configs:\n    print(f\"\\nTest: batch={batch}, seq={seq}, d_model={d_model}, heads={heads}\")\n    print(\"-\" * 70)\n    \n    # Créer input\n    x_test = torch.randn(batch, seq, d_model)\n    \n    # Créer module\n    mha_test = MultiHeadAttention(d_model=d_model, num_heads=heads)\n    \n    # Forward pass\n    output_test = mha_test(x_test)\n    \n    # Vérifier\n    print(f\"  Input shape:  {x_test.shape}\")\n    print(f\"  Output shape: {output_test.shape}\")\n    print(f\"  ✓ Dimension préservée: {x_test.shape == output_test.shape}\")\n    \n    # Vérifier les poids d'attention\n    weights_test = mha_test.get_attention_weights(x_test)\n    print(f\"  Attention weights shape: {weights_test.shape}\")\n    print(f\"  ✓ Expected: ({batch}, {heads}, {seq}, {seq})\")

## 6. Exercices Pratiques\n\n### Exercice 1: Expérimenter avec Différents Nombres de Têtes\n\nTestez multi-head attention avec 1, 2, 4, et 8 têtes. Observez comment les patterns changent.

In [ ]:
# TODO: Exercice 1 - Expérimenter avec différents nombres de têtes\n\nd_model_ex = 16\nseq_len_ex = 5\nhead_counts = [1, 2, 4, 8]\n\ntorch.manual_seed(42)\nx_ex = torch.randn(1, seq_len_ex, d_model_ex)\n\nprint(\"Comparaison de différents nombres de têtes:\")\nprint(\"=\" * 70)\n\nfor num_heads_ex in head_counts:\n    print(f\"\\nNombre de têtes: {num_heads_ex}\")\n    print(\"-\" * 70)\n    \n    # Créer le module\n    mha_ex = MultiHeadAttention(d_model=d_model_ex, num_heads=num_heads_ex)\n    \n    # Forward pass\n    output_ex = mha_ex(x_ex)\n    \n    # Statistiques\n    params = sum(p.numel() for p in mha_ex.parameters())\n    print(f\"  d_k (par tête): {d_model_ex // num_heads_ex}\")\n    print(f\"  Paramètres: {params}\")\n    print(f\"  Output shape: {output_ex.shape}\")\n\nprint(\"\\n✓ Observation: Plus de têtes = plus de paramètres = plus de capacité\")

## 7. Résumé\n\n### Ce que nous avons appris\n\n1. **Formule du Multi-Head Attention:**\n   - $\\text{MultiHead}(Q, K, V) = \\text{Concat}(\\text{head}_1, ..., \\text{head}_h)W^O$\n   - Chaque tête calcule son attention indépendamment\n\n2. **Les 5 étapes:**\n   - Projections linéaires (Q, K, V)\n   - Division en têtes (reshape + transpose)\n   - Attention par tête (parallèle)\n   - Concaténation des têtes\n   - Projection finale\n\n3. **Avantages:**\n   - **Diversité:** Chaque tête se spécialise\n   - **Parallélisme:** Calcul efficace\n   - **Capacité:** Plus de paramètres\n   - **Robustesse:** Redondance\n\n4. **Dimension préservée:**\n   - Input: (batch, seq_len, d_model)\n   - Output: (batch, seq_len, d_model)\n   - Propriété importante des transformers!\n\n### Prochaines étapes\n\nDans le prochain notebook, nous assemblerons tous ces composants pour créer un TransformerBlock complet!